terminal: https://python.land/virtual-environments/virtualenv

yingliu@Yings-Air fraud detection in healthcare % source healthfraud/bin/activate

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing


#### Read data 

In [3]:
# data train include provider ID and labels
Train = pd.read_csv("data/Train-1542865627584.csv")

# include beneficiary KYC details like health conditions,region
Train_Beneficiarydata = pd.read_csv("data/Train_Beneficiarydata-1542865627584.csv")

# include the claims filed for those patients who are admitted in the hospitals
Train_Inpatientdata=pd.read_csv("data/Train_Inpatientdata-1542865627584.csv")

# include the claims filed for those patients who visit hospitals and not admitted in it.
Train_Outpatientdata=pd.read_csv("data/Train_Outpatientdata-1542865627584.csv")


In [23]:
print("Train\n", Train.shape)
print("Train_Beneficiarydata\n", Train_Beneficiarydata.shape)
print("Train_Inpatientdata\n", Train_Inpatientdata.shape)
print("Train_Outpatientdata\n", Train_Outpatientdata.shape)


Train
 (5410, 2)
Train_Beneficiarydata
 (138556, 25)
Train_Inpatientdata
 (40474, 30)
Train_Outpatientdata
 (517737, 27)


#### Exploratory Data Analysis 

##### Inpatient

###### The most common procedure code and diagnosis code 

In [27]:
# figure out the most common procedure code and diagnosis code for inpatient cases
print('inpatient columns:\n',  Train_Inpatientdata.columns)
Train_Inpatientdata.head(5)

inpatient columns:
 Index(['BeneID', 'ClaimID', 'ClaimStartDt', 'ClaimEndDt', 'Provider',
       'InscClaimAmtReimbursed', 'AttendingPhysician', 'OperatingPhysician',
       'OtherPhysician', 'AdmissionDt', 'ClmAdmitDiagnosisCode',
       'DeductibleAmtPaid', 'DischargeDt', 'DiagnosisGroupCode',
       'ClmDiagnosisCode_1', 'ClmDiagnosisCode_2', 'ClmDiagnosisCode_3',
       'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5', 'ClmDiagnosisCode_6',
       'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8', 'ClmDiagnosisCode_9',
       'ClmDiagnosisCode_10', 'ClmProcedureCode_1', 'ClmProcedureCode_2',
       'ClmProcedureCode_3', 'ClmProcedureCode_4', 'ClmProcedureCode_5',
       'ClmProcedureCode_6'],
      dtype='object')


,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,...,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6
0,BENE11001,CLM46614,2009-04-12,2009-04-18,PRV55912,26000,PHY390922,NaN,NaN,2009-04-12,...,2724,19889,5849,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BENE11001,CLM66048,2009-08-31,2009-09-02,PRV55907,5000,PHY318495,PHY318495,NaN,2009-08-31,...,NaN,NaN,NaN,NaN,7092.0,NaN,NaN,NaN,NaN,NaN
2,BENE11001,CLM68358,2009-09-17,2009-09-20,PRV56046,5000,PHY372395,NaN,PHY324689,2009-09-17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BENE11011,CLM38412,2009-02-14,2009-02-22,PRV52405,5000,PHY369659,PHY392961,PHY349768,2009-02-14,...,25062,40390,4019,NaN,331.0,NaN,NaN,NaN,NaN,NaN
4,BENE11014,CLM63689,2009-08-13,2009-08-30,PRV56614,10000,PHY379376,PHY398258,NaN,2009-08-13,...,5119,29620,20300,NaN,3893.0,NaN,NaN,NaN,NaN,NaN


In [28]:
# create a dataframe for concatinating all Procedure Code columns 
inpatient_proc = pd.DataFrame(columns = ['ProcedureCode'])
inpatient_proc['ProcedureCode'] = pd.concat([Train_Inpatientdata["ClmProcedureCode_1"], 
                                           Train_Inpatientdata["ClmProcedureCode_2"],
                                           Train_Inpatientdata["ClmProcedureCode_3"],
                                           Train_Inpatientdata["ClmProcedureCode_4"], 
                                           Train_Inpatientdata["ClmProcedureCode_5"], 
                                           Train_Inpatientdata["ClmProcedureCode_6"]], axis=0, sort=True).dropna()

print(inpatient_proc.shape)
inpatient_proc.head(5)

(29692, 1)


,ProcedureCode
1,7092.0
3,331.0
4,3893.0
5,863.0
6,4576.0


In [30]:
# importing the module

 
# creating the DataFrames
df1 = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'], 
                    'B': ['B0', 'B1', 'B2', 'B3']})
display('df1:', df1)
df2 = pd.DataFrame({'C': ['C0', 'C1', 'C2', 'C3'], 
                    'D': ['D0', 'D1', 'D2', 'D3']})
display('df2:', df2)
 
# concatenating
display('After concatenating:')
display(pd.concat([df1, df2],
                  axis = 1))
display(pd.concat([df1, df2],
                  axis = 0))
                  

'df1:'

,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3


'df2:'

,C,D
0,C0,D0
1,C1,D1
2,C2,D2
3,C3,D3


'After concatenating:'

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3


,A,B,C,D
0,A0,B0,NaN,NaN
1,A1,B1,NaN,NaN
2,A2,B2,NaN,NaN
3,A3,B3,NaN,NaN
0,NaN,NaN,C0,D0
1,NaN,NaN,C1,D1
2,NaN,NaN,C2,D2
3,NaN,NaN,C3,D3


In [31]:
pd.concat([Train_Inpatientdata["ClmProcedureCode_1"], 
                                           Train_Inpatientdata["ClmProcedureCode_2"],
                                           Train_Inpatientdata["ClmProcedureCode_3"],
                                           Train_Inpatientdata["ClmProcedureCode_4"], 
                                           Train_Inpatientdata["ClmProcedureCode_5"], 
                                           Train_Inpatientdata["ClmProcedureCode_6"]], axis=0, sort=True)

0           NaN
1        7092.0
2           NaN
3         331.0
4        3893.0
          ...  
40469       NaN
40470       NaN
40471       NaN
40472       NaN
40473       NaN
Length: 242844, dtype: float64

In [32]:
pd.concat([Train_Inpatientdata["ClmDiagnosisCode_1"], 
                                           Train_Inpatientdata["ClmDiagnosisCode_2"],
                                           Train_Inpatientdata["ClmDiagnosisCode_3"],
                                           Train_Inpatientdata["ClmDiagnosisCode_4"], 
                                           Train_Inpatientdata["ClmDiagnosisCode_5"], 
                                           Train_Inpatientdata["ClmDiagnosisCode_6"]], axis=0, sort=True)

0         1970
1         6186
2        29623
3        43491
4          042
         ...  
40469     5173
40470    V1042
40471     4111
40472    42822
40473      412
Length: 242844, dtype: object

#### Merging OutPatient ,Inpatient and Beneficiary to get the whole dataset


In [5]:
Train_Allpatientdata = pd.merge(Train_Outpatientdata, Train_Inpatientdata, how='outer')
Train_Allpatientdata = pd.merge(Train_Allpatientdata, Train_Beneficiarydata, on = 'BeneID', how = 'inner')

#### Data preprocessing

In [6]:
# add labels
df = pd.merge(Train_Allpatientdata, Train, on = 'Provider')
df

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmDiagnosisCode_1,...,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,PotentialFraud
0,BENE11002,CLM624349,2009-10-11,2009-10-11,PRV56011,30,PHY326117,NaN,NaN,78943,...,2,2,2,2,2,0,0,30,50,Yes
1,BENE11004,CLM121801,2009-01-06,2009-01-06,PRV56011,40,PHY334319,NaN,NaN,71988,...,1,1,1,1,2,0,0,1810,760,Yes
2,BENE11004,CLM150998,2009-01-22,2009-01-22,PRV56011,200,PHY403831,NaN,NaN,82382,...,1,1,1,1,2,0,0,1810,760,Yes
3,BENE11004,CLM173224,2009-02-03,2009-02-03,PRV56011,20,PHY339887,NaN,NaN,20381,...,1,1,1,1,2,0,0,1810,760,Yes
4,BENE11004,CLM224741,2009-03-03,2009-03-03,PRV56011,40,PHY345721,NaN,NaN,V6546,...,1,1,1,1,2,0,0,1810,760,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558206,BENE57978,CLM37910,2009-02-11,2009-02-16,PRV53511,0,PHY378514,PHY426843,NaN,40491,...,1,1,2,1,2,2200,2136,30,100,No
558207,BENE80245,CLM79653,2009-12-15,2009-12-18,PRV51711,4000,PHY345109,NaN,NaN,2948,...,1,2,2,2,2,4000,1068,0,0,No
558208,BENE85177,CLM77745,2009-11-29,2009-12-07,PRV54503,8000,PHY418343,NaN,NaN,515,...,2,1,2,2,2,8000,1068,0,0,No
558209,BENE105577,CLM53155,2009-05-28,2009-05-31,PRV51239,5000,PHY382916,NaN,NaN,27651,...,1,1,2,2,2,5000,1068,0,0,No


In [7]:
# Drop irrelevant columns
df = df.drop(['BeneID', 'ClaimID'], axis = 1)

In [8]:
#show null values amount
print('null value amount:\n', df.isna().sum())

#show null values percentage
print('null value percentage:\n', df.isnull().mean()*100)

#figure out high percentage columns
nullPercent = df.isnull().mean()*100
filtered_columns = nullPercent[nullPercent > 70].index
print(filtered_columns)



null value amount:
 ClaimStartDt                            0
ClaimEndDt                              0
Provider                                0
InscClaimAmtReimbursed                  0
AttendingPhysician                   1508
OperatingPhysician                 443764
OtherPhysician                     358475
ClmDiagnosisCode_1                  10453
ClmDiagnosisCode_2                 195606
ClmDiagnosisCode_3                 315156
ClmDiagnosisCode_4                 393675
ClmDiagnosisCode_5                 446287
ClmDiagnosisCode_6                 473819
ClmDiagnosisCode_7                 492034
ClmDiagnosisCode_8                 504767
ClmDiagnosisCode_9                 516396
ClmDiagnosisCode_10                553201
ClmProcedureCode_1                 534901
ClmProcedureCode_2                 552721
ClmProcedureCode_3                 557242
ClmProcedureCode_4                 558093
ClmProcedureCode_5                 558202
ClmProcedureCode_6                 558211
DeductibleAmtP

In [10]:
# deal with dx code and proc code
# Applying coalesce function to dx columns 
# filling dx_1 by other dx columns  
df['ClmDiagnosisCode_1'] = df[[f'ClmDiagnosisCode_{i}' for i in range(1, 11)]].fillna(method='ffill', axis=1)['ClmDiagnosisCode_1']
df['ClmDiagnosisCode_2'] = df[[f'ClmDiagnosisCode_{i}' for i in range(2, 11)]].fillna(method='ffill', axis=1)['ClmDiagnosisCode_2']
df['ClmDiagnosisCode_3'] = df[[f'ClmDiagnosisCode_{i}' for i in range(3, 11)]].fillna(method='ffill', axis=1)['ClmDiagnosisCode_3']


# Applying coalesce function to proc columns 
df['ClmProcedureCode_1'] = df[[f'ClmProcedureCode_{i}' for i in range(1, 7)]].fillna(method='ffill', axis=1)['ClmProcedureCode_1']
df['ClmProcedureCode_2'] = df[[f'ClmProcedureCode_{i}' for i in range(2, 7)]].fillna(method='ffill', axis=1)['ClmProcedureCode_2']



In [11]:
#show null values amount
print('null value amount:\n', df.isna().sum())

#show null values percentage
print('null value percentage:\n', df.isnull().mean()*100)

#figure out high percentage columns
nullPercent = df.isnull().mean()*100
filtered_columns = nullPercent[nullPercent > 70].index
print(filtered_columns)

null value amount:
 ClaimStartDt                            0
ClaimEndDt                              0
Provider                                0
InscClaimAmtReimbursed                  0
AttendingPhysician                   1508
OperatingPhysician                 443764
OtherPhysician                     358475
ClmDiagnosisCode_1                  10453
ClmDiagnosisCode_2                 195606
ClmDiagnosisCode_3                 315156
ClmDiagnosisCode_4                 393675
ClmDiagnosisCode_5                 446287
ClmDiagnosisCode_6                 473819
ClmDiagnosisCode_7                 492034
ClmDiagnosisCode_8                 504767
ClmDiagnosisCode_9                 516396
ClmDiagnosisCode_10                553201
ClmProcedureCode_1                 534901
ClmProcedureCode_2                 552721
ClmProcedureCode_3                 557242
ClmProcedureCode_4                 558093
ClmProcedureCode_5                 558202
ClmProcedureCode_6                 558211
DeductibleAmtP

In [12]:
# three methods to deal with dx columns and proc columns
# method 1: drop columns where null values percentage > 70%
# remove columns which null value percentage is larger than 70% after processing
df = df.drop(['ClmDiagnosisCode_4', 'ClmDiagnosisCode_5',
       'ClmDiagnosisCode_6', 'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8',
       'ClmDiagnosisCode_9', 'ClmDiagnosisCode_10', 'ClmProcedureCode_1',
       'ClmProcedureCode_2', 'ClmProcedureCode_3', 'ClmProcedureCode_4',
       'ClmProcedureCode_5', 'ClmProcedureCode_6'], axis = 1)

In [ ]:
# method 2: concatenate all the dx code and apply one-hot encoding (don't consider the seq)



In [ ]:
# method 3: concatenate all the dx code and apply one-hot encoding (consider the seq)



In [13]:
# calculate the number of inpatient days
startdate = pd.to_datetime(df.AdmissionDt)
enddate = pd.to_datetime(df.DischargeDt)
df['Hperiod'] = (enddate - startdate).dt.days

In [14]:
# calculate the period of claim
startdate = pd.to_datetime(df.ClaimStartDt)
enddate = pd.to_datetime(df.ClaimEndDt)
df['Cperiod'] = (enddate - startdate).dt.days

check for same physician in all three physician columns

In [15]:
def physician_same(row):
    atten_oper = row['AttendingPhysician'] == row['OperatingPhysician']
    oper_other = row['OperatingPhysician'] == row['OtherPhysician']
    atten_other = row['AttendingPhysician'] == row['OtherPhysician']
    
    # atten = oper = other
    if atten_oper == True and oper_other == True:
        return 0
    
    # atten = oper != other
    elif atten_oper == True and oper_other == False:
        return 1
    
    # atten = other != oper
    elif atten_other == True and oper_other == False:
        return 2
    
    # atten != other = oper
    elif atten_other == False and oper_other == True:
        return 3
    
    # atten != other != oper
    else:
        return 4


df['phy_same'] = df.apply(physician_same, axis=1)

encode label and check whether the data is balanced

['No', 'Yes']


array([1, 0])

In [22]:
# check whether the data is balanced
# 38% positive cases
df['PotentialFraud'].unique()
(df['PotentialFraud'] == "Yes").sum() /len(df)




0.3812106891480103

one hot encoding for the categorical features

In [56]:
from sklearn.preprocessing import OneHotEncoder     # One Hot Encoding 

encoder = OneHotEncoder()
one_hot = encoder.fit_transform(df[['Gender']]).toarray()

# df = pd.concat([df, pd.DataFrame(one_hot, columns=encoder.get_feature_names())], axis=1)


# print(df)